# Preparation

In [ ]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'../..')

from trafficSimulator import *
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

## Simulation setup

### Simulation params setup

In [ ]:
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

vehicle_rate = 60
fast_track_factor, slow_track_factor = 1, 0.33
stop_distance = 25
fast_wait_time = 4
slow_wait_time = 4
vehicle_limit = 80

df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [ ]:
def parallel_line(current_coordinates, distance):
    """
    Returns the coordinate sets of lines that are parallel to the given line
    with the speicified distance
    
    Input:
        current_coordinates: two set of x and y coordinates that mark the start
                             and end of the given line
        distance: the distance between the given line and the parallel lines to
                  to be output
    """
    
    x1, y1 = current_coordinates[0]
    x2, y2 = current_coordinates[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    
    cross_x = -b/k
    hypo = math.sqrt(b**2+cross_x**2)
    
    if k>0:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))

        new_coord_2 = ((x2-distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
        
    else:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
    
        new_coord_2 = ((x2-distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))
    
    return new_coord_1, new_coord_2

In [ ]:
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

bottom_left_inbound = parallel_line(current_coordinates=left_bottom_outbound, 
                                    distance=2.5)[0]
right_bottom_inbound = parallel_line(current_coordinates=bottom_right_outbound, 
                                     distance=2.5)[1]

top_left_inbound = parallel_line(current_coordinates=left_top_outbound, 
                                 distance=2.5)[0]
right_top_inbound = parallel_line(current_coordinates=top_right_outbound, 
                                  distance=2.5)[1]

## For the connections of the topology, we can either represent with concrete lines
## or just as a dot (as shown below). Either case, the vehicles will not actually go 
## pass these routes; they are more for demonstration purposes

connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [ ]:
# Smaller delta encourages more exploration
delta = 0.25

# Penalty proportion for delay caused
alpha = 7.5

# Interpolation factor between money and time
gamma_mean, gamma_var = 7.5, 0.75

# Level of risk aversion
eta_mean, eta_var = 1, 0.2

## Function needed for running simulation

In [ ]:
 def run_simulation(round_number, all_routes, vehicle_limit, df, vehicle_preferences,
                    vehicle_rate, vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    global bottom_left_inbound
    global right_bottom_inbound
    global top_left_inbound
    global right_top_inbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        # A dictionary that stores the vehicles' labels as keys and their list of probabilities
        # of choosing each possible route as values
        # For the initial few rounds all vehicles choose all routes with equal weights.
        # For more complicated topologies some other algorithms will be needed as all possible
        # routes are not as explicit as in here.
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes

        # Outbound corners
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [ ]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 10s, sys: 6.15 s, total: 3min 16s
Wall time: 3min 11s


In [ ]:
record_df1

Vehicle_label Road_order Total_time Leading_vehicles  \
0              0     [0, 3]     104.15       [-999, 13]   
1              1     [0, 1]       84.3          [0, 21]   
2              2     [2, 3]      67.75     [-999, -999]   
3              3     [0, 1]      89.25          [1, 22]   
4              4     [0, 1]      88.65           [3, 3]   
..           ...        ...        ...              ...   
75            75     [2, 1]     166.15         [74, 74]   
76            76     [2, 1]      168.3         [75, 75]   
77            77     [0, 3]      181.0         [72, 69]   
78            78     [0, 3]      182.9         [77, 77]   
79            79     [2, 3]      200.4         [76, 73]   

               Stopped_time         Stop_while_front  
0                    [0, 0]  [2.4499999999999993, 0]  
1    [5.849999999999987, 0]   [5.849999999999987, 0]  
2                    [0, 0]                   [0, 0]  
3   [2.4499999999999993, 0]  [2.4499999999999993, 0]  
4                    [0, 0]                   [0, 0]  
..                      ...                      ...  
75  [32.250000000000306, 0]                   [0, 0]  
76   [34.85000000000016, 0]                   [0, 0]  
77  [2.6999999999999984, 0]                   [0, 0]  
78   [3.949999999999994, 0]                   [0, 0]  
79   [37.84999999999999, 0]                   [0, 0]  

[80 rows x 6 columns]

## Simulation Analysis of the 1st round

In [ ]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

Vehicle_label Road_order Total_time Leading_vehicles  \
0             0     [0, 3]     104.15       [-999, 13]   
1             1     [0, 1]       84.3          [0, 21]   
2             2     [2, 3]      67.75     [-999, -999]   
3             3     [0, 1]      89.25          [1, 22]   
4             4     [0, 1]      88.65           [3, 3]   
5             5     [0, 3]      122.8          [4, 26]   
6             6     [0, 3]      124.2           [5, 5]   
7             7     [2, 1]      36.55        [2, -999]   
8             8     [2, 3]      69.15           [7, 2]   
9             9     [2, 1]      39.35           [8, 7]   

               Stopped_time         Stop_while_front  Round_number  
0                    [0, 0]  [2.4499999999999993, 0]             1  
1    [5.849999999999987, 0]   [5.849999999999987, 0]             1  
2                    [0, 0]                   [0, 0]             1  
3   [2.4499999999999993, 0]  [2.4499999999999993, 0]             1  
4                    [0, 0]                   [0, 0]             1  
5   [2.0500000000000007, 0]  [2.0500000000000007, 0]             1  
6  [0.49999999999999994, 0]                   [0, 0]             1  
7                    [0, 0]                   [0, 0]             1  
8                    [0, 0]                   [0, 0]             1  
9                    [0, 0]                   [0, 0]             1

### Set up dataframe

In [ ]:
utility_df = pd.DataFrame(columns=['Vehicle_label', 'Utilities', 'Probabilities'])

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

In [ ]:
etas = []
gammas = []

for i in range(len(utility_df)):
    eta = np.random.normal(eta_mean, eta_var, 1)[0]
    while eta < 0:
        eta = np.random.normal(eta_mean, eta_var, 1)[0]
    etas.append(eta)
    
for i in range(len(utility_df)):
    gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    while gamma < 0:
        gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    gammas.append(gamma)

utility_df['Gamma'] = gammas
utility_df['Eta'] = etas

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

Vehicle_label     Utilities Probabilities Routes_taken     Gamma       Eta
0              0  [0, 0, 0, 0]  [0, 0, 0, 0]           []  6.614726  0.904946
1              1  [0, 0, 0, 0]  [0, 0, 0, 0]           []  7.752500  0.957298
2              2  [0, 0, 0, 0]  [0, 0, 0, 0]           []  8.031307  1.130883
3              3  [0, 0, 0, 0]  [0, 0, 0, 0]           []  7.133938  1.103313
4              4  [0, 0, 0, 0]  [0, 0, 0, 0]           []  8.552510  0.785395

## Summary of the 1st round

In [ ]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

Vehicle_label  Road_order  Caused_delay  Count  Total_time
0              0           0          5.85      1      104.15
1              1           1          2.45      1       84.30
2              2           2          0.00      1       67.75
3              3           1          0.00      1       89.25
4              4           1          2.05      1       88.65
5              5           0          0.50      1      122.80
6              6           0          0.00      1      124.20
7              7           3          0.00      1       36.55
8              8           2          0.00      1       69.15
9              9           3          0.00      1       39.35

Vehicle_label                                          Utilities  \
0              0  [-6.378973107567525, 16.651092223153956, 16.65...   
1              1  [16.651092223153956, -6.734785892999831, 16.65...   
2              2  [16.651092223153956, 16.651092223153956, -8.11...   
3              3  [16.651092223153956, -7.514981538239647, 16.65...   
4              4  [16.651092223153956, -3.939251225660181, 16.65...   
5              5  [-8.238712170693702, 16.651092223153956, 16.65...   
6              6  [-7.5673168501697905, 16.651092223153956, 16.6...   
7              7  [16.651092223153956, 16.651092223153956, 16.65...   
8              8  [16.651092223153956, 16.651092223153956, -10.7...   
9              9  [16.651092223153956, 16.651092223153956, 16.65...   

                                       Probabilities Routes_taken     Gamma  \
0  [3.319314891035254e-11, 0.33333333332226894, 0...          [0]  6.614726   
1  [0.3333333333255816, 2.3255245304506384e-11, 0...          [1]  7.752500   
2  [0.33333333333139226, 0.33333333333139226, 5.8...          [2]  8.031307   
3  [0.3333333333297806, 1.0658258800501832e-11, 0...          [1]  7.133938   
4  [0.33333333320642644, 3.807204497268035e-10, 0...          [1]  8.552510   
5  [5.168613307069255e-12, 0.33333333333161047, 0...          [0]  8.950564   
6  [1.0114800567615055e-11, 0.33333333332996173, ...          [0]  7.145042   
7  [0.33333333084977135, 0.33333333084977135, 0.3...          [3]  6.503963   
8  [0.33333333333319404, 0.33333333333319404, 4.1...          [2]  7.310229   
9  [0.3333333333330151, 0.3333333333330151, 0.333...          [3]  7.203186   

        Eta  
0  0.904946  
1  0.957298  
2  1.130883  
3  1.103313  
4  0.785395  
5  1.054727  
6  1.086393  
7  0.689277  
8  1.201125  
9  1.230150

In [ ]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [ ]:
%%time

import time
time_l = []
begin = time.time()

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 100

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)
    
    time_l.append(round(time.time()-begin-sum(time_l), 2))

CPU times: user 5h 52min 8s, sys: 8min 37s, total: 6h 46s
Wall time: 5h 50min 36s


In [ ]:
utility_tmp.to_csv(f"Saved_data/Simulation_records/Summary.csv", index=False)